In [1]:
import openpathsampling as paths
from openpathsampling.experimental.storage import Storage, monkey_patch_all
paths = monkey_patch_all(paths)

from tqdm.auto import tqdm

from openpathsampling.analysis.new_filters import *

In [2]:
storage = Storage("./oneway.db", mode='r')

First, we can use these filters directly on steps:

In [14]:
for step in RejectedSteps(storage.steps):
    ...  # here is where you might do something to analyze rejection reasons

In [16]:
# we can use some basic Python to get a quick count of accepted steps
len(list(AcceptedSteps(storage.steps)))

559

We can also use the step filters in combination with sample filters:

In [ ]:
replica_0_filter = Replica(0)
for step in RejectedSteps(storage.steps):
    samples = step.change.canonical.trials
    for sample in replica_0_filter(samples):
        ... # do something with each sample

However, since this is really common, we have a class `SampleFilter` that does that for you. Note that we have 3 primary stages: filtering based on the steps (`step_filter`), then selecting samples from those steps (`sample_selector`), then filtering based on the samples (`sample_filter`).

In [18]:
filt = SampleFilter(step_filter=RejectedSteps, 
                    sample_selector=TrialSamples,
                    sample_filter=Replica(0))

for sample in filt(steps, flatten=True):
    ... # just like above, but a little cleaner

The `flatten` argument says to give each sample. The default behavior is for each step to return a list of samples (which may be an empty list for some steps).

Finally, the part that's actually pretty neat -- we can use logical combinations of the filters:

In [19]:
filt = AcceptedSteps | RejectedSteps
len(list(filt(storage.steps)))

1001

In [21]:
filt = RejectedSteps & CanonicalMover('ForwardShootMover')
len(list(filt(storage.steps)))

153

These can be arbitrarily complex (just remember to use parentheses correctly!):

In [23]:
filt = (
    (RejectedSteps & CanonicalMover('ForwardShootMover'))
    | (AcceptedSteps & CanonicalMover('BackwardShootMover'))
)
len(list(filt(storage.steps)))

337

In [24]:
# bad parens here give inherent contradiction
filt = (
    RejectedSteps 
    & (CanonicalMover('ForwardShootMover') | AcceptedSteps)
    & CanonicalMover('BackwardShootMover')
)
len(list(filt(storage.steps)))

0

So far, if you combine different stages, you'll get an error.

In [25]:
RejectedSteps & TrialSamples & Replica(0)

TypeError: Cannot combine filters <openpathsampling.analysis.new_filters.StepFilter object at 0x7fdd03b20410> and <openpathsampling.analysis.new_filters.SampleSelector object at 0x7fdd03b20250>

In [ ]:
# TODO: need to add reprs to make that a little easier to read